##### Setup & Imports

In [1]:
import os
import sys
from pathlib import Path
import shutil
from typing import List, Tuple, Dict
import numpy as np
import cv2
import yaml
from datetime import datetime
from tqdm import tqdm
import json

from ultralytics import YOLO
from ultralytics.utils import LOGGER
import torch

print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

print("✓ All imports successful")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

2.9.1+cu130
13.0
True
NVIDIA GeForce RTX 3060 Laptop GPU
✓ All imports successful
✓ CUDA available: True


##### Configuration & Path Setup

In [ ]:
# Configuration
CONFIG = {
    # EXISTING ROBOFLOW OBB DATASET (ALREADY SPLIT)
    'dataset_root': './Birds-detect.v16i.yolov8-obb',  
    
    # NEGATIVE SAMPLES (TO BE SPLIT)
    'negative_images_dir': './negative-sky-data/train/images', 
    
    # OUTPUT (MERGED DATASET)
    'output_dataset_root': './Birds-detect.v16i.yolov8-obb',  

    # Default and unmodified OBB dataset
    'default_dataset': './default-Birds-detect.v16i.yolov8-obb',
    
    # MODEL CONFIGURATION
    'model_name': 'yolo11n-obb',
    'input_size': 512,
    'num_classes': 1,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
}

print("✓ Configuration loaded")
print(f"Dataset root: {CONFIG['dataset_root']}")
print(f"Negative samples: {CONFIG['negative_images_dir']}")
print(f"Default dataset: {CONFIG['default_dataset']}")



✓ Configuration loaded
Dataset root: ./Birds-detect.v16i.yolov8-obb
Negative samples: ./negative-sky-data/train/images
Default dataset: ./default-Birds-detect.v16i.yolov8-obb


##### ANALYZE EXISTING DATASET SPLIT RATIO

In [3]:
# Calculate train/valid/test split ratio from your current dataset.
# This ratio will be used to split the negative samples.

def get_split_ratio(dataset_root: str) -> Dict[str, float]:
    """
    Analyze existing dataset and calculate split ratios.
    Expected structure:
        dataset/
        ├── train/images/
        ├── valid/images/
        └── test/images/
    """
    dataset_path = Path(dataset_root)
    
    splits = {
        'train': 0,
        'valid': 0,
        'test': 0
    }
    
    split_counts = {
        'train': 0,
        'valid': 0,
        'test': 0
    }
    
    # Count images in each split
    image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.PNG'}
    
    for split in ['train', 'valid', 'test']:
        images_dir = dataset_path / split / 'images'
        if images_dir.exists():
            image_files = [f for f in images_dir.glob('*') if f.suffix in image_extensions]
            split_counts[split] = len(image_files)
    
    # Calculate ratios
    total = sum(split_counts.values())
    for split in splits:
        splits[split] = split_counts[split] / total if total > 0 else 0
    
    return splits, split_counts, total

# Calculate ratios
split_ratio, split_counts, total_existing = get_split_ratio(CONFIG['dataset_root'])

print("\n" + "=" * 70)
print("EXISTING DATASET ANALYSIS")
print("=" * 70)
for split, count in split_counts.items():
    ratio = split_ratio[split]
    print(f"{split}:")
    print(f"  Images: {count}")
    print(f"  Ratio: {ratio:.2%}")

print(f"\nTotal existing samples: {total_existing}")
print("=" * 70)



EXISTING DATASET ANALYSIS
train:
  Images: 3463
  Ratio: 86.02%
valid:
  Images: 258
  Ratio: 6.41%
test:
  Images: 305
  Ratio: 7.58%

Total existing samples: 4026


##### DISCOVER & COUNT NEGATIVE SAMPLES

In [4]:
# 4. Discover Negative Samples
# Find and catalog all negative sample images to be split.

def discover_negative_samples(negative_images_dir: str) -> List[str]:
    """
    Discover all negative sample images.
    These will be split according to the dataset ratio.
    """
    neg_dir = Path(negative_images_dir)
    negative_images = []
    image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.PNG'}
    
    if neg_dir.exists():
        for img_file in neg_dir.glob('*'):
            if img_file.suffix in image_extensions:
                negative_images.append(str(img_file))
    else:
        print(f"⚠ Warning: Negative images directory not found: {neg_dir}")
    
    return negative_images

# Discover negative samples
negative_images = discover_negative_samples(CONFIG['negative_images_dir'])

print(f"\nNegative Samples Found:")
print(f"  Total: {len(negative_images)}")
if negative_images:
    print(f"  Examples:")
    for img in negative_images[:3]:
        print(f"    - {Path(img).name}")
print("=" * 70)



Negative Samples Found:
  Total: 326
  Examples:
    - Ac-N001_jpg.rf.b73506468695f3dd753f096c88b7ff80.jpg
    - Ac-N002_jpg.rf.49e8265d7286aad5622b200194730c65.jpg
    - Ac-N003_jpg.rf.9f20e35b20f58d4a058b9bf87c73f237.jpg


##### SPLIT NEGATIVE SAMPLES ACCORDING TO RATIO

In [5]:
def split_negative_samples(negative_images: List[str], 
                          split_ratio: Dict[str, float]) -> Dict[str, List[str]]:
    """
    Split negative samples according to the existing dataset ratio.
    Example: if dataset is 70% train, 15% valid, 15% test
             then negatives will be split the same way.
    """
    # Shuffle for random distribution
    np.random.seed(42)
    shuffled_negatives = negative_images.copy()
    np.random.shuffle(shuffled_negatives)
    
    n_total = len(shuffled_negatives)
    
    # Calculate split indices
    splits = {}
    current_idx = 0
    
    for split in ['train', 'valid', 'test']:
        ratio = split_ratio[split]
        count = int(n_total * ratio)
        
        splits[split] = shuffled_negatives[current_idx:current_idx + count]
        current_idx += count
    
    # Add any remaining to test split (due to rounding)
    remaining = shuffled_negatives[current_idx:]
    splits['test'].extend(remaining)
    
    return splits

# Split the negative samples
negative_splits = split_negative_samples(negative_images, split_ratio)

print("\n" + "=" * 70)
print("NEGATIVE SAMPLES SPLIT")
print("=" * 70)

for split in ['train', 'valid', 'test']:
    count = len(negative_splits[split])
    ratio = count / len(negative_images) * 100
    print(f"{split}:")
    print(f"  Count: {count}")
    print(f"  Ratio: {ratio:.1f}%")

print(f"\nTotal negatives to add: {sum(len(v) for v in negative_splits.values())}")
print("=" * 70)


NEGATIVE SAMPLES SPLIT
train:
  Count: 280
  Ratio: 85.9%
valid:
  Count: 20
  Ratio: 6.1%
test:
  Count: 26
  Ratio: 8.0%

Total negatives to add: 326


##### CREATE EMPTY LABELS FOR NEGATIVES

In [ ]:
# For each negative image, create an empty .txt label file.
# Empty label = "This image has no objects to detect

def create_empty_labels_for_negatives(image_path: str) -> str:
    """
    Create empty .txt label file for a negative sample image.
    """
    img_file = Path(image_path)
    label_file = img_file.parent / (img_file.stem + '.txt')
    
    if not label_file.exists():
        label_file.touch()  # Create empty file
    
    return str(label_file)

print("\nCreating empty label files for negatives...")
print("-" * 70)

for split in ['train', 'valid', 'test']:
    print(f"\nProcessing {split} split ({len(negative_splits[split])} images)...")
    
    for img_path in tqdm(negative_splits[split], desc=f"  Creating labels"):
        create_empty_labels_for_negatives(img_path)

print("\n" + "-" * 70)
print("Empty label files created for all negative samples")
print("=" * 70)


Creating empty label files for negatives...
----------------------------------------------------------------------

Processing train split (280 images)...


  Creating labels: 100%|██████████| 280/280 [00:00<00:00, 10961.10it/s]



Processing valid split (20 images)...


  Creating labels: 100%|██████████| 20/20 [00:00<00:00, 11161.00it/s]



Processing test split (26 images)...


  Creating labels: 100%|██████████| 26/26 [00:00<00:00, 12704.09it/s]


----------------------------------------------------------------------
✓ Empty label files created for all negative samples


In [7]:
# 7. Copy Negatives to Dataset Structure
# Copy negative images and their corresponding empty label files to the dataset.

def copy_negatives_to_dataset(dataset_root: str, negative_splits: Dict[str, List[str]]):
    """
    Copy negative images and empty labels to dataset structure.
    """
    dataset_path = Path(dataset_root)
    
    print("\nCopying negative samples to dataset...")
    print("-" * 70)
    
    for split in ['train', 'valid', 'test']:
        images_dir = dataset_path / split / 'images'
        labels_dir = dataset_path / split / 'labels'
        
        # Ensure directories exist
        images_dir.mkdir(parents=True, exist_ok=True)
        labels_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\nProcessing {split} split ({len(negative_splits[split])} negatives)...")
        
        for img_src in tqdm(negative_splits[split], desc=f"  Copying to {split}"):
            img_src_path = Path(img_src)
            
            # Copy image
            img_dst = images_dir / img_src_path.name
            if not img_dst.exists():
                shutil.copy2(img_src, img_dst)
            
            # Copy/create empty label
            lbl_src = img_src_path.parent / (img_src_path.stem + '.txt')
            lbl_dst = labels_dir / (img_src_path.stem + '.txt')
            
            if lbl_src.exists() and not lbl_dst.exists():
                shutil.copy2(str(lbl_src), str(lbl_dst))
            elif not lbl_dst.exists():
                # Create empty label if it doesn't exist
                lbl_dst.touch()
    
    print("\n" + "-" * 70)
    print("✓ All negative samples copied to dataset")
    print("=" * 70)

# Execute copy operation
copy_negatives_to_dataset(CONFIG['dataset_root'], negative_splits)


Copying negative samples to dataset...
----------------------------------------------------------------------

Processing train split (280 negatives)...


  Copying to train: 100%|██████████| 280/280 [00:00<00:00, 3111.51it/s]



Processing valid split (20 negatives)...


  Copying to valid: 100%|██████████| 20/20 [00:00<00:00, 3209.23it/s]



Processing test split (26 negatives)...


  Copying to test: 100%|██████████| 26/26 [00:00<00:00, 3578.29it/s]


----------------------------------------------------------------------
✓ All negative samples copied to dataset


##### VALIDATION & SUMMARY (with negatives)

In [ ]:
# 8. Final Validation & Summary
# Verify the integration is complete and correct.


def final_validation(dataset_root: str) -> bool:
    """
    Validate that negatives were properly integrated.
    """
    print("\n" + "=" * 70)
    print("FINAL VALIDATION")
    print("=" * 70)
    
    dataset_path = Path(dataset_root)
    all_good = True
    
    for split in ['train', 'valid', 'test']:
        images_dir = dataset_path / split / 'images'
        labels_dir = dataset_path / split / 'labels'
        
        if not (images_dir.exists() and labels_dir.exists()):
            print(f"✗ {split}: Missing images or labels directory")
            all_good = False
            continue
        
        img_count = len(list(images_dir.glob('*')))
        lbl_count = len(list(labels_dir.glob('*.txt')))
        
        match = img_count == lbl_count
        print(f"{'✓' if match else '✗'} {split}: {img_count} images, {lbl_count} labels")
        
        if not match:
            all_good = False
            
        # Count empty labels (negatives)
        empty_count = sum(1 for f in labels_dir.glob('*.txt') 
                         if f.stat().st_size == 0)
        if empty_count > 0:
            print(f"  └─ Contains {empty_count} negative samples (empty labels)")
    
    print("\n" + "=" * 70)
    if all_good:
        print("✓ ALL VALIDATION CHECKS PASSED!")
        print("Integration complete and ready for training")
    else:
        print("✗ Some checks failed. Review above.")
    print("=" * 70)
    
    return all_good

# Execute validation
is_valid = final_validation(CONFIG['dataset_root'])


# Print final summary
print("\n" + "=" * 70)
print("INTEGRATION SUMMARY")
print("=" * 70)

print(f"\nDataset Statistics:")
print(f"   Existing positive samples: {total_existing}")
print(f"   New negative samples: {len(negative_images)}")
print(f"   Total samples after integration: {total_existing + len(negative_images)}")

print(f"\nSplit Distribution (with negatives added):")
for split in ['train', 'valid', 'test']:
    print(f"   {split}: {split_counts[split]} existing + {len(negative_splits[split])} new = {split_counts[split] + len(negative_splits[split])}")

print(f"\nDataset Location: {CONFIG['dataset_root']}")
print(f"\n Ready for training!")
print(f"   Use existing data.yaml or training script")
print("\n" + "=" * 70)




FINAL VALIDATION
✓ train: 3463 images, 3463 labels
  └─ Contains 442 negative samples (empty labels)
✓ valid: 258 images, 258 labels
  └─ Contains 38 negative samples (empty labels)
✓ test: 305 images, 305 labels
  └─ Contains 35 negative samples (empty labels)

✓ ALL VALIDATION CHECKS PASSED!
Integration complete and ready for training

INTEGRATION SUMMARY

📊 Dataset Statistics:
   Existing positive samples: 4026
   New negative samples: 326
   Total samples after integration: 4352

📊 Split Distribution (with negatives added):
   train: 3463 existing + 280 new = 3743
   valid: 258 existing + 20 new = 278
   test: 305 existing + 26 new = 331

📁 Dataset Location: ./Birds-detect.v16i.yolov8-obb

✓ Ready for training!
   Use existing data.yaml or training script



##### Verify data.yaml (with negative data)

In [9]:
# 9. Verify data.yaml Configuration
# Check that your data.yaml is correct for training.

def display_data_yaml(dataset_root: str):
    """
    Display the data.yaml configuration.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    
    print("\n" + "=" * 70)
    print("DATA.YAML CONFIGURATION")
    print("=" * 70)
    
    if yaml_path.exists():
        with open(yaml_path, 'r') as f:
            content = f.read()
        print(content)
        print("\n" + "=" * 70)
        print("✓ data.yaml is ready for training")
    else:
        print(f"⚠ data.yaml not found at: {yaml_path}")
        print("You can use your existing data.yaml or create a new one")
    
    print("=" * 70)

# Display configuration
display_data_yaml(CONFIG['dataset_root'])



DATA.YAML CONFIGURATION
train: train/images
val: valid/images
test: test/images

names: 
  0: bird

✓ data.yaml is ready for training


#### Verify data.yaml (default data)

In [5]:
# 9. Verify data.yaml Configuration
# Check that your data.yaml is correct for training.

def display_data_yaml(dataset_root: str):
    """
    Display the data.yaml configuration.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    
    print("\n" + "=" * 70)
    print("DATA.YAML CONFIGURATION")
    print("=" * 70)
    
    if yaml_path.exists():
        with open(yaml_path, 'r') as f:
            content = f.read()
        print(content)
        print("\n" + "=" * 70)
        print("✓ data.yaml is ready for training")
    else:
        print(f"⚠ data.yaml not found at: {yaml_path}")
        print("You can use your existing data.yaml or create a new one")
    
    print("=" * 70)

# Display configuration
display_data_yaml(CONFIG['default_dataset'])


DATA.YAML CONFIGURATION
train: train/images
val: valid/images
test: test/images

names: 
  0: bird

✓ data.yaml is ready for training


##### Train model (with negative samples)

In [ ]:


def train_model(dataset_root: str, config: Dict):
    """
    Train YOLOv11n-OBB with integrated dataset and Learning Rate Decay.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    
    if not yaml_path.exists():
        print(f"Error: data.yaml not found at {yaml_path}")
        return None, None
    
    print("\n" + "=" * 70)
    print(f"STARTING TRAINING")
    print("=" * 70)
    print(f"Dataset: {dataset_root}")
    print(f"Epochs: 150")
    print(f"Batch size: 16")
    print(f"Device: {config['device']}")
    print("=" * 70 + "\n")
    
    # Load pretrained model
    model = YOLO(f"{config['model_name']}.pt")
    
    # Train
    results = model.train(
        data=str(yaml_path),
        epochs=150,
        imgsz=config['input_size'],
        batch=16,

        workers=0,
        
        # --- LEARNING RATE SETTINGS ---
        lr0=0.001,              # Initial Learning Rate (1e-3 is standard for AdamW)
        lrf=0.01,               # Final Learning Rate Fraction (Final LR = lr0 * lrf)
        cos_lr=True,            # Enable Cosine Annealing Learning Rate Decay
        warmup_epochs=3.0,      # Warmup epochs before decay starts
        # -----------------------------
        
        device=config['device'],
        project='./results',
        name='bird_detection_with_negatives',
        patience=20,
        save=True,
        verbose=True,
        
        # OBB-specific augmentations
        mosaic=1.0,
        mixup=0.5,
        degrees=180,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
    )
    
    print("\n" + "=" * 70)
    print("TRAINING COMPLETE")
    print("=" * 70)
    
    return model, results

model, results = train_model(CONFIG['dataset_root'], CONFIG)


STARTING TRAINING
Dataset: ./Birds-detect.v16i.yolov8-obb
Epochs: 150
Batch size: 16
Device: cuda

New https://pypi.org/project/ultralytics/8.3.240 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.237  Python-3.13.5 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=Birds-detect.v16i.yolov8-obb\data.yaml, degrees=180, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300,

#### Train model (default)

In [ ]:
def train_model(dataset_root: str, config: Dict):
    """
    Train YOLOv11n-OBB with integrated dataset and Learning Rate Decay.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    
    if not yaml_path.exists():
        print(f"Error: data.yaml not found at {yaml_path}")
        return None, None
    
    print("\n" + "=" * 70)
    print(f"STARTING TRAINING")
    print("=" * 70)
    print(f"Dataset: {dataset_root}")
    print(f"Epochs: 150")
    print(f"Batch size: 16")
    print(f"Device: {config['device']}")
    print("=" * 70 + "\n")
    
    # Load pretrained model
    model = YOLO(f"{config['model_name']}.pt")
    
    # Train
    results = model.train(
        data=str(yaml_path),
        epochs=150,
        imgsz=config['input_size'],
        batch=16,

        workers=0,
        
        # --- LEARNING RATE SETTINGS ---
        lr0=0.001,              # Initial Learning Rate (1e-3 is standard for AdamW)
        lrf=0.01,               # Final Learning Rate Fraction (Final LR = lr0 * lrf)
        cos_lr=True,            # Enable Cosine Annealing Learning Rate Decay
        warmup_epochs=3.0,      # Warmup epochs before decay starts
        # -----------------------------
        
        device=config['device'],
        project='./default_results',
        name='bird_detection_without_negatives',
        patience=20,
        save=True,
        verbose=True,
        
        # OBB-specific augmentations
        mosaic=1.0,
        mixup=0.5,
        degrees=180,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
    )
    
    print("\n" + "=" * 70)
    print("TRAINING COMPLETE")
    print("=" * 70)
    
    return model, results

model, results = train_model(CONFIG['default_dataset'], CONFIG)


STARTING TRAINING
Dataset: ./default-Birds-detect.v16i.yolov8-obb
Epochs: 150
Batch size: 16
Device: cuda

New https://pypi.org/project/ultralytics/8.3.240 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.237  Python-3.13.5 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=default-Birds-detect.v16i.yolov8-obb\data.yaml, degrees=180, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio

##### Evaluate on best test set

In [ ]:
# 11. Evaluate Model
# Evaluate the trained model on the test set.

def evaluate_model(dataset_root: str, model_path: str, config: Dict):
    """
    Evaluate model on test set.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    
    print("\n" + "=" * 70)
    print("FINAL TEST SET EVALUATION")
    print("=" * 70)
    
    # Load best model
    best_model = YOLO(model_path)
    
    # Validate on test split
    metrics = best_model.val(
        data=str(yaml_path),
        split='test',
        device=config['device']
    )
    
    # Print results
    print("\n" + "-" * 70)
    print("FINAL RESULTS")
    print("-" * 70)
    print(f"mAP@0.5:    {metrics.box.map50:.4f}")
    print(f"mAP@0.5:95: {metrics.box.map:.4f}")
    print(f"Precision:  {metrics.box.mp:.4f}")
    print(f"Recall:     {metrics.box.mr:.4f}")
    print("-" * 70)
    
    return metrics

# Uncomment after training completes
model_path = './results/bird_detection_with_negatives3/weights/best.pt'
metrics = evaluate_model(CONFIG['dataset_root'], model_path, CONFIG)



FINAL TEST SET EVALUATION
Ultralytics 8.3.237  Python-3.13.5 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n-obb summary (fused): 109 layers, 2,653,918 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 211.973.1 MB/s, size: 49.4 KB)
val: Scanning C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detection\Birds-detect.v16i.yolov8-obb\test\labels.cache... 305 images, 35 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 305/305 271.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 20/20 3.3it/s 6.0s0.2s
                   all        305        849      0.822       0.85      0.881      0.523
Speed: 1.9ms preprocess, 2.6ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detecti

#### Evaluate without negative data model

In [8]:
# 11. (Optional) Evaluate Model
# Evaluate the trained model on the test set.

def evaluate_model(dataset_root: str, model_path: str, config: Dict):
    """
    Evaluate model on test set.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    
    print("\n" + "=" * 70)
    print("FINAL TEST SET EVALUATION")
    print("=" * 70)
    
    # Load best model
    best_model = YOLO(model_path)
    
    # Validate on test split
    metrics = best_model.val(
        data=str(yaml_path),
        split='test',
        device=config['device'],
        workers=0
    )
    
    # Print results
    print("\n" + "-" * 70)
    print("FINAL RESULTS")
    print("-" * 70)
    print(f"mAP@0.5:    {metrics.box.map50:.4f}")
    print(f"mAP@0.5:95: {metrics.box.map:.4f}")
    print(f"Precision:  {metrics.box.mp:.4f}")
    print(f"Recall:     {metrics.box.mr:.4f}")
    print("-" * 70)
    
    return metrics

# Uncomment after training completes
model_path = './default_results/bird_detection_without_negatives/weights/best.pt'
metrics = evaluate_model(CONFIG['default_dataset'], model_path, CONFIG)


FINAL TEST SET EVALUATION
Ultralytics 8.3.237  Python-3.13.5 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n-obb summary (fused): 109 layers, 2,653,918 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 150.170.5 MB/s, size: 40.4 KB)
val: Scanning C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detection\default-Birds-detect.v16i.yolov8-obb\test\labels.cache... 279 images, 9 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 279/279 438.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 18/18 3.9it/s 4.6s0.2s
                   all        279        849      0.834      0.856       0.88      0.539
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird 

#### Test model without negatives on data that has negatives

In [5]:
from typing import Dict

def evaluate_model(dataset_root: str, model_path: str, config: Dict):
    """
    Evaluate model on test set.
    """
    yaml_path = Path(dataset_root) / 'data.yaml'
    real_world_dataset_yaml = './Birds-detect.v16i.yolov8-obb/data.yaml' 
    
    print("\n" + "=" * 70)
    print("FINAL TEST SET EVALUATION")
    print("=" * 70)
    
    # Load best model
    best_model = YOLO(model_path)
    
    # Validate on test split
    metrics = best_model.val(
        data=str(real_world_dataset_yaml),
        split='test',
        device=config['device'],
        workers=0
    )
    
    # Print results
    print("\n" + "-" * 70)
    print("FINAL RESULTS")
    print("-" * 70)
    print(f"mAP@0.5:    {metrics.box.map50:.4f}")
    print(f"mAP@0.5:95: {metrics.box.map:.4f}")
    print(f"Precision:  {metrics.box.mp:.4f}")
    print(f"Recall:     {metrics.box.mr:.4f}")
    print("-" * 70)
    
    return metrics

# Uncomment after training completes
model_path = './default_results/bird_detection_without_negatives/weights/best.pt'
metrics = evaluate_model(CONFIG['default_dataset'], model_path, CONFIG)


FINAL TEST SET EVALUATION
Ultralytics 8.3.237  Python-3.13.5 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n-obb summary (fused): 109 layers, 2,653,918 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 5.32.3 MB/s, size: 42.0 KB)
val: Scanning C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detection\Birds-detect.v16i.yolov8-obb\test\labels.cache... 305 images, 35 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 305/305 197.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 20/20 2.5it/s 8.1s0.4s
                   all        305        849      0.828      0.856      0.878      0.537
Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detection\